### This notebook is initial submission for the hackathon 

https://datahack.analyticsvidhya.com/contest/janatahack-customer-segmentation/

My public and private leaderboard scores are 0.9314285714 and 0.9251743817 respectively

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../data/Train_aBjfeNk.csv')
test = pd.read_csv('../data/Test_LqhgPWU.csv')
sample_submission = pd.read_csv('../submissions/sample_submission_wyi0h0z.csv')

In [16]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
ID                 8068 non-null int64
Gender             8068 non-null object
Ever_Married       7928 non-null object
Age                8068 non-null int64
Graduated          7990 non-null object
Profession         7944 non-null object
Work_Experience    7239 non-null float64
Spending_Score     8068 non-null object
Family_Size        7733 non-null float64
Var_1              7992 non-null object
Segmentation       8068 non-null object
dtypes: float64(2), int64(2), object(7)
memory usage: 693.4+ KB


In [14]:
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 10 columns):
ID                 2627 non-null int64
Gender             2627 non-null object
Ever_Married       2577 non-null object
Age                2627 non-null int64
Graduated          2603 non-null object
Profession         2589 non-null object
Work_Experience    2358 non-null float64
Spending_Score     2627 non-null object
Family_Size        2514 non-null float64
Var_1              2595 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 205.3+ KB


In [5]:
sample_submission.head()

,ID,Segmentation
0,458989,A
1,458994,A
2,458996,A
3,459000,A
4,459001,A


### Distribution of data

#### Spending score

In [19]:
train.Spending_Score.value_counts()

Low        4878
Average    1974
High       1216
Name: Spending_Score, dtype: int64

In [20]:
test.Spending_Score.value_counts()

Low        1616
Average     625
High        386
Name: Spending_Score, dtype: int64

#### Profession

In [21]:
train.Profession.value_counts()

Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: Profession, dtype: int64

In [22]:
test.Profession.value_counts()

Artist           802
Healthcare       418
Entertainment    301
Doctor           242
Engineer         236
Lawyer           221
Executive        176
Marketing        111
Homemaker         82
Name: Profession, dtype: int64

#### Graduation

In [25]:
train.Graduated.value_counts()

Yes    4968
No     3022
Name: Graduated, dtype: int64

In [26]:
test.Graduated.value_counts()

Yes    1602
No     1001
Name: Graduated, dtype: int64

#### Work experience

In [23]:
train.Work_Experience.value_counts()

1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: Work_Experience, dtype: int64

In [24]:
test.Work_Experience.value_counts()

1.0     773
0.0     769
8.0     149
9.0     139
4.0      93
2.0      87
3.0      82
5.0      76
6.0      61
7.0      60
14.0     21
11.0     14
12.0     12
10.0     11
13.0     11
Name: Work_Experience, dtype: int64

#### var distribution

In [64]:
train.Var_1.value_counts()

Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: Var_1, dtype: int64

In [65]:
test.Var_1.value_counts()

Cat_6    1672
Cat_4     386
Cat_3     267
Cat_2     141
Cat_7      66
Cat_1      34
Cat_5      29
Name: Var_1, dtype: int64

### Filling Nan values with appropriate values

In [8]:
target_df = train[['Segmentation']]

In [33]:
all_data = pd.concat([train.drop(columns=['Segmentation']),
           test], axis=0).reset_index(drop=True)

In [41]:
(train.drop(columns=['Segmentation']) == all_data.loc[:8067]).ID.value_counts()

True    8068
Name: ID, dtype: int64

In [46]:
(test==all_data.loc[8068:].reset_index(drop=True)).ID.value_counts()

True    2627
Name: ID, dtype: int64

In [48]:
def spending_score(row):
    if row=='Low':
        return 0
    elif row=='Average':
        return 1
    else:
        return 2

In [52]:
all_data['Spending_Score'] = all_data['Spending_Score'].apply(spending_score)

In [51]:
all_data.Spending_Score.isna().value_counts()

False    10695
Name: Spending_Score, dtype: int64

In [55]:
all_data['Work_Experience'] = all_data.Work_Experience.fillna(-1)

In [56]:
all_data.Work_Experience.isna().value_counts()

False    10695
Name: Work_Experience, dtype: int64

In [58]:
all_data['Family_Size'] = all_data.Family_Size.fillna(-1)

In [59]:
all_data.Family_Size.isna().value_counts()

False    10695
Name: Family_Size, dtype: int64

In [62]:
all_data['Var_1'] = all_data.Var_1.fillna('Cat_Unknown')

In [63]:
all_data.Var_1.isna().value_counts()

False    10695
Name: Var_1, dtype: int64

In [67]:
all_data['Profession'] = all_data.Profession.fillna('Profession_Unknown')

In [68]:
all_data.Profession.isna().value_counts()

False    10695
Name: Profession, dtype: int64

In [73]:
def yes_no_fill(row):
    if row=='Yes':
        return 1
    elif row=='No':
        return 0
    else:
        return row

In [71]:
all_data['Ever_Married'] = all_data.Ever_Married.fillna(-1)

In [72]:
all_data['Ever_Married'].value_counts()

Yes    6163
No     4342
-1      190
Name: Ever_Married, dtype: int64

In [75]:
all_data['Ever_Married'] = all_data['Ever_Married'].apply(yes_no_fill)

In [79]:
all_data['Graduated'].value_counts()

Yes    6570
No     4023
Name: Graduated, dtype: int64

In [82]:
all_data['Graduated'] = all_data.Graduated.fillna(-1)

In [83]:
all_data['Graduated'] = all_data['Graduated'].apply(yes_no_fill)

In [84]:
all_data.Graduated.isna().value_counts()

False    10695
Name: Graduated, dtype: int64

In [85]:
all_data.Graduated.value_counts()

 1    6570
 0    4023
-1     102
Name: Graduated, dtype: int64

In [86]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 10 columns):
ID                 10695 non-null int64
Gender             10695 non-null object
Ever_Married       10695 non-null int64
Age                10695 non-null int64
Graduated          10695 non-null int64
Profession         10695 non-null object
Work_Experience    10695 non-null float64
Spending_Score     10695 non-null int64
Family_Size        10695 non-null float64
Var_1              10695 non-null object
dtypes: float64(2), int64(5), object(3)
memory usage: 835.6+ KB


In [100]:
data_model_ready = pd.concat([all_data,
            pd.concat([pd.get_dummies(all_data.Profession), 
                      pd.concat([pd.get_dummies(all_data.Gender),      
                                 pd.get_dummies(all_data.Var_1)],
                                axis=1)],
                     axis=1)],
          axis=1).drop(columns=['Gender', 'Profession', 'Var_1'])

### Model Building and Submissions

In [106]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [170]:
def create_submission_file(target_list, file_name):
    submission_df = pd.concat([X_predict[['ID']],
           pd.DataFrame({'Segmentation': target_list})], axis=1)
    submission_df.to_csv('../submissions/'+file_name+'.csv', index=False)

In [103]:
target = pd.get_dummies(target_df)

In [166]:
X_input = data_model_ready[:8068]
X_predict = data_model_ready[8068:].reset_index(drop=True)

In [204]:
rfc = RandomForestClassifier()
rfc.fit(X=X_input, y=target_df)
rfc.score(X=X_input, y=target_df)

0.9775656916212196

In [ ]:
create_submission_file(rfc.predict(X_predict), 'rf_classifier')